# HW1 - Applied Quantitative Logistics

## Definitions

**Brute Force:** straightforward methods of solving a problem that rely on sheer computing power and trying every possibility rather than advanced techniques to improve efficiency. [link](https://textbooks.cs.ksu.edu/cc310/4-data-structures-and-algorithms/12-brute-force/#:~:text=Simply%20put%2C%20a%20brute%20force,over%20and%20try%20the%20other.)

- A brute force algorithm solves a problem through exhaustion: it goes through all possible choices until a solution is found.

- The time complexity of a brute force algorithm is often proportional to the input size.

- Brute force algorithms are simple and consistent, but very slow.

Instruction for submission:

- Please submit your solutions in (.ipynb) format to my email (msohrabi@hse.ru).

- Deadline: **February 3, 2023, 11:59 pm.**

- The subject of the email: **[HW1_AQL]-YOUR_NAME**

In [33]:
import pandas as pd



### Requirements

In [34]:
df_resources = pd.DataFrame([[1, "A"], [2, "B"], [3, "C"],
                             [4, "D"],[5, "E"]], columns=["ID", "Name"])
df_resources

,ID,Name
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [35]:
df_projects = pd.DataFrame([[1, "a", "10.05.2020", "15.05.2020"], [2, "b", "13.05.2020", "27.07.2020"],
                            [3, "c", "08.07.2020", "30.07.2020"], [4, "d", "11.12.2020", "29.12.2020"],
                            [5, "e", "06.11.2020", "07.11.2020"]], 
                           columns=["ID", "Name", "Start_Time", "End_Time"])
                           
df_projects

,ID,Name,Start_Time,End_Time
0,1,a,10.05.2020,15.05.2020
1,2,b,13.05.2020,27.07.2020
2,3,c,08.07.2020,30.07.2020
3,4,d,11.12.2020,29.12.2020
4,5,e,06.11.2020,07.11.2020


In [36]:
df_expertise = pd.DataFrame([[1, 1, 1], [2, 5, 3], [3, 2, 4], [4, 4, 5], [5, 3, 2], [6, 2, 1], [7, 3, 1], [8, 2, 2]],
                           columns=["ID", "ID_res", "ID_pro"])

df_expertise

,ID,ID_res,ID_pro
0,1,1,1
1,2,5,3
2,3,2,4
3,4,4,5
4,5,3,2
5,6,2,1
6,7,3,1
7,8,2,2


### All Feasible Combinations

In [37]:
#using python dictionary to find the combinations
#ID_pro as keys and ID_res as values
mapping = dict()
for index, row in df_expertise.iterrows():
    #print(row['ID_pro'], row['ID_res'])
    if row['ID_pro'] not in mapping:
        mapping[row['ID_pro']] = [row['ID_res']]
    else:
        mapping[row['ID_pro']].append(row['ID_res'])

#sorting based on ID_pro
new_mapping = {}
for i in sorted(mapping.keys()):
    new_mapping[i] = mapping[i]
print(new_mapping)



{1: [1, 2, 3], 2: [3, 2], 3: [5], 4: [2], 5: [4]}


In [38]:
import itertools as it
#finding all possible combinations of resources
resources = list(it.product(*new_mapping.values()))
print("Possible combinations of resources:", resources)
map_projects = []
for sol in resources:
    proj = [(i, j) for i, j in zip(new_mapping.keys(), sol)]
    map_projects.append(proj)
print("#########################  Allocation of resources after mapping with Projects  ###################################")
#first item represents project id and 2nd represents resource
for i,aloc in enumerate(map_projects):
    print("Allocated mapped resource {0}: {1}".format(i+1,aloc))

Possible combinations of resources: [(1, 3, 5, 2, 4), (1, 2, 5, 2, 4), (2, 3, 5, 2, 4), (2, 2, 5, 2, 4), (3, 3, 5, 2, 4), (3, 2, 5, 2, 4)]
#########################  Allocation of resources after mapping with Projects  ###################################
Allocated mapped resource 1: [(1, 1), (2, 3), (3, 5), (4, 2), (5, 4)]
Allocated mapped resource 2: [(1, 1), (2, 2), (3, 5), (4, 2), (5, 4)]
Allocated mapped resource 3: [(1, 2), (2, 3), (3, 5), (4, 2), (5, 4)]
Allocated mapped resource 4: [(1, 2), (2, 2), (3, 5), (4, 2), (5, 4)]
Allocated mapped resource 5: [(1, 3), (2, 3), (3, 5), (4, 2), (5, 4)]
Allocated mapped resource 6: [(1, 3), (2, 2), (3, 5), (4, 2), (5, 4)]


### Brute Force - [Algorithm Name]

In [39]:
from datetime import datetime, time
import collections
def date_in_Seconds(dt1):
    date1 = datetime.strptime(dt1, '%d.%m.%Y')
    return date1

df_projects['Start_Time'] = df_projects.apply(lambda x: date_in_Seconds(x.Start_Time), axis=1)
df_projects['End_Time'] = df_projects.apply(lambda x: date_in_Seconds(x.End_Time), axis=1)
df_projects




,ID,Name,Start_Time,End_Time
0,1,a,2020-05-10,2020-05-15
1,2,b,2020-05-13,2020-07-27
2,3,c,2020-07-08,2020-07-30
3,4,d,2020-12-11,2020-12-29
4,5,e,2020-11-06,2020-11-07


In [40]:
#store combinations with no overlap
no_overlap = []
#check project ID for the corresponding resource ID
def check_projects(map,dup_val):
    check = []
    for i,(proj,res) in enumerate(map):
        if res == dup_val:
            check.append(proj)
    return check


In [41]:
for map in map_projects:
    resources = [res for proj, res in map]
    #find duplicates in the resource
    duplicates = [item for item, count in collections.Counter(resources).items() if count > 1]
    flag = 1
    for dup in duplicates:
        #find corresponding projects
        check_prefix = check_projects(map,dup)
        for i in range(len(check_prefix)):
            for j in range(i+1, len(check_prefix)):
                #find start and end date of projects
                st1= df_projects.loc[df_projects['ID'] == check_prefix[i], ['Start_Time','End_Time']].iloc[0][0]
                et1 = df_projects.loc[df_projects['ID'] == check_prefix[i], ['Start_Time','End_Time']].iloc[0][1]
                st2= df_projects.loc[df_projects['ID'] == check_prefix[j], ['Start_Time','End_Time']].iloc[0][0]
                et2 = df_projects.loc[df_projects['ID'] == check_prefix[j], ['Start_Time','End_Time']].iloc[0][1]
                
                st1,et1 = datetime.strptime(str(st1),"%Y-%m-%d %H:%M:%S"), datetime.strptime(str(et1),"%Y-%m-%d %H:%M:%S")
                st2,et2 = datetime.strptime(str(st2),"%Y-%m-%d %H:%M:%S"), datetime.strptime(str(et2),"%Y-%m-%d %H:%M:%S")

                #compare dates
                
                if (st1 <= et2) & (st2<= et1):
                        
                    flag = 0
                    break 
    
    if flag:
        #update non overlapped solutions
        no_overlap.append(map)

### Print the Solutions

In [42]:
#printing all the non overlapped solutions
#inside the tuple first element represents proj_ID and 2nd element represents res_ID
for i,sol in enumerate(no_overlap):
    print("Non overlapped solution {0}: {1}".format(i+1,sol)) 

Non overlapped solution 1: [(1, 1), (2, 3), (3, 5), (4, 2), (5, 4)]
Non overlapped solution 2: [(1, 1), (2, 2), (3, 5), (4, 2), (5, 4)]
Non overlapped solution 3: [(1, 2), (2, 3), (3, 5), (4, 2), (5, 4)]
Non overlapped solution 4: [(1, 3), (2, 2), (3, 5), (4, 2), (5, 4)]
